### 

# <center> Interpretação dos parâmetros de regressão linear com dados transformados

___

## Objetivo

Este material irá demonstrar como a unidade de medida dos dados pode alterar a interpretação dos parâmetros em modelos de regressão linear. 

Temos vasto material sobre as aplicações e importância da transformação dos dados na construção de modelos de machine learning. Muitos algoritimos já esperam receber os dados em um formato específico, outros são flexíveis, cabendo ao analista aplicar e testar as transformações para encontrar o melhor modelo.

Desde que não precisemos interpretar individualmente como cada variável afeta a variável resposta, o entendimento sobre o impacto da transformação nos dados pode ser secundário. Caso seja necessário maior interpretabilidade do modelo, precisamos compreender de que forma essas transformções afetam a interpretação do parâmetros.

## Descrição dos dados

O dataset contém os dados de consumo de bens duráveis, não duráveis, consumo de serviços e despesas totais de consumo pessoal por trimestre nos EUA medidos em bilhões entre os anos de 2003 e 2006.

Mais informações sobre este e outros modelos podem ser econtrados no livro Econometria básica de Damodar e Porter no capítulo 6. Os dados podem ser obtidos na tabela 6.3.


Descrição:
* QUARTER: Ano e trimestre
* EXPSERVICES: Gastos com serviços
* EXPDUR: Gastos com bens duráveis
* EXPNONDUR: Gastos com bens não duráveis
* PCEXP: Despesas totais com consumo pessoal

Vamos explorar os modelos a seguir observando o comportamento dos gastos com bens duráveis em relação aos gastos totais.

* LinLin - Variáveis independentes (X) e dependente (y) nominais (escala/formato original)
* LinLog - Variáveis independentes transformadas (Log(X)) e dependente (y) nominal.
* LogLin - Variáveis independentes nominais (X) e dependente transformada (Log(y)).
* LogLog - Variáveis independentes e dependente com transformação log.
* LinLin Recíproco - Variáveis independentes 1/x
* Dados padronizados - Média 0 e desvio-padrão 1.

Também vamos explorar a taxa de crescimento dos gastos com serviços no tempo.

___

# Pacotes e dados

In [1]:
# Importar pacotes
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
# Importar dados
dados = pd.read_excel('dados/Damodar_Dataset/Table 6_3.xls', skiprows = 4, nrows = 14)

df = dados.copy()

In [3]:
# Visualizar
df

,YEAR-QUARTER,EXPSERVICES,EXPDUR,EXPNONDUR,PCEXP
0,2003-I,4143.3,971.4,2072.5,7184.9
1,2003-II,4161.3,1009.8,2084.2,7249.3
2,2003-III,4190.7,1049.6,2123.0,7352.9
3,2003-IV,4220.2,1051.4,2132.5,7394.3
4,2004-I,4268.2,1067.0,2155.3,7479.8
5,2004-II,4308.4,1071.4,2164.3,7534.4
6,2004-III,4341.5,1093.9,2184.0,7607.1
7,2004-IV,4377.4,1110.3,2213.1,7687.1
8,2005-I,4395.3,1116.8,2241.5,7739.4
9,2005-II,4420.0,1150.8,2268.4,7819.8


___

# Análise exploratória

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   YEAR-QUARTER  14 non-null     object 
 1   EXPSERVICES   14 non-null     float64
 2   EXPDUR        14 non-null     float64
 3   EXPNONDUR     14 non-null     float64
 4   PCEXP         14 non-null     float64
dtypes: float64(4), object(1)
memory usage: 688.0+ bytes


# Processamento 

O dataset dispensa qualquer tratamento. Vamos apenas adicionar a variável TIME e fazer algumas transformações que serão úteis mais a frente.

In [5]:
# Selecionar as variáveis de interesse
df = df[['YEAR-QUARTER','EXPDUR', 'PCEXP', 'EXPSERVICES']]

# Adicionar var tempo
df['TIME'] = range(1,15,1)

# Adicionar EXPDUR e PCEXP padronizados
df['EXPDUR_STD'] = df['EXPDUR'].apply(lambda x: (x - df['EXPDUR'].mean())/df['EXPDUR'].std())
df['PCEXP_STD'] = df['PCEXP'].apply(lambda x: (x - df['PCEXP'].mean())/df['PCEXP'].std())
df['1/PCEXP'] = df['PCEXP'].apply(lambda x: (1/x))

# Adicionar EXPDUR e PCEXP log
df['EXPDUR_LOG'] = df['EXPDUR'].apply(lambda x: (np.log(x)))
df['PCEXP_LOG'] = df['PCEXP'].apply(lambda x: (np.log(x)))

# View 
df

,YEAR-QUARTER,EXPDUR,PCEXP,EXPSERVICES,TIME,EXPDUR_STD,PCEXP_STD,1/PCEXP,EXPDUR_LOG,PCEXP_LOG
0,2003-I,971.4,7184.9,4143.3,1,-1.911090,-1.606924,0.000139,6.878738,8.879737
1,2003-II,1009.8,7249.3,4161.3,2,-1.336288,-1.377853,0.000138,6.917508,8.888660
2,2003-III,1049.6,7352.9,4190.7,3,-0.740529,-1.009348,0.000136,6.956164,8.902850
3,2003-IV,1051.4,7394.3,4220.2,4,-0.713585,-0.862089,0.000135,6.957878,8.908465
4,2004-I,1067.0,7479.8,4268.2,5,-0.480071,-0.557966,0.000134,6.972606,8.919961
5,2004-II,1071.4,7534.4,4308.4,6,-0.414209,-0.363754,0.000133,6.976721,8.927234
6,2004-III,1093.9,7607.1,4341.5,7,-0.077410,-0.105160,0.000131,6.997505,8.936837
7,2004-IV,1110.3,7687.1,4377.4,8,0.168078,0.179400,0.000130,7.012386,8.947299
8,2005-I,1116.8,7739.4,4395.3,9,0.265376,0.365430,0.000129,7.018223,8.954079
9,2005-II,1150.8,7819.8,4420.0,10,0.774316,0.651413,0.000128,7.048213,8.964414


___

# Transformações logarítmicas, recíproca e padronização

Vamos regredir os gastos com bens duráveis (EXPDUR) contra as despesas totais com consumo pessoal (PCEXP), aplicar as transformações e interpretar cada resutado.

## Trasformações logarítmicas

A transformação log gera alguns benefícios de grande interesse como a linearização de  modelos não lineares e a obtenção da elasticidade ou semi-elasticidade entre das variáveis.

**Tomemos o modelo não linear como exemplo:**

### $ Y_i = \beta_{0}X_i^{\beta_1}e^{u_i} $

Aplicando log temos:

### $ LnY_i = Ln\beta_{0} + \beta_1LnX_i + u_i $
### $ LnY_i = \alpha + \beta_1LnX_i + u_i $

O modelo agora é linear e a relação entre X e y é a elasticidade de y em relação X. Dada uma variação percentual em X, qual seria a variação percentual observada em y? Estamos falando da variação percentual das variáveis e não mais da variação nominal.

A combinação de regressor na escala natural e regressando na escala logarítmica e vice-versa, gera modelos interessantes por demonstrar a semi-elasticidades entre as variáveis, ou seja, qual o efeito nominal sobre uma variável dada uma varição percentual em outra variável.

### Modelo LinLin

Tanto as variáveis independentes como a dependente estão no seu formato/escala original.

In [6]:
# Definir X e y
X = df['PCEXP']
y = df['EXPDUR']

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.params

const   -684.302854
PCEXP      0.233528
dtype: float64

> ### Interpretação

Como o parâmetro da variável PCEXP sugere, a variação de 1 unidade na variável PCEXP causa uma variação de 0.2335 unidade na variável EXPDUR. Ou seja, para cada mil dólares gastos a mais no consumo total (PCEXP), espera-se em média um aumento de $233,5 com gastos em bens duráveis (EXPDUR). 

### Modelo LinLog

A variável dependente (y) está no seu formato/escala original e a variável independente será transformada (Ln(X)).

In [7]:
# Definir X e y
X = np.log(df['PCEXP'])
y = df['EXPDUR']

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.params

const   -14818.334419
PCEXP     1780.453541
dtype: float64

> ### Interpretação

O modelo LinLog mede a variação da variável y na escala original dos dados dada uma variação % em X.
No nosso modelo, espera-se que ocorra em média o aumento de $1.780 nos gastos com bens duráveis (EXPDUR) para cada 1% de aumento nos gastos totais com consumo (PCEXP).

### Modelo LogLin

A variável dependente será transformada (Ln(y)) e a variável independente permanece no seu formato/escala original.

In [8]:
# Definir X e y
X = df['PCEXP']
y = np.log(df['EXPDUR'])

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.params

const    5.364949
PCEXP    0.000214
dtype: float64

> ### Interpretação

Demonstra a variação de % na variável y dada uma variação nominal na variável X.

Trazendo para o nosso modelo, temos que para cada dólar gasto a mais com gastos totais com consumo (PCEXP), podemos esperar em média um aumento de 0.0214% nos gastos com bens duráveis (EXPDUR).

### Modelo LogLog

Tanto a variável dependente (EXPDUR) quanto a variável independente (PCEXP) estão transformadas.

In [9]:
# Definir X e y
X = np.log(df['PCEXP'])
y = np.log(df['EXPDUR'])

# Modelo
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.params

const   -7.604271
PCEXP    1.633626
dtype: float64

> ### Interpretação

Demonstra a variação de % na variável y dada uma variação % na variável X.

O nosso modelo sugere que para um aumento de 1% nos gastos totais com consumo (PCEXP), espera-se em média um aumento de 1.63% nos gastos com bens duráveis (EXPDUR).

## Variáveis LinLin Recíproco $1/X$

Para discutir o caso recíproco e mantermos o meso conjunto de dados estamos forçando a hipótese de uma relação inversa entre os gastos totais e gastos com bens duráveis. Em outro material discutimos esse modelo explorando dados de desemprego e inflação por meio da curva de Phillips.

In [10]:
# Definir X e y
X = df['1/PCEXP']
y = df['EXPDUR']

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
params = lin_reg.params.apply(lambda x: float(x))
print('Const ',params[0].round(2))
print('1/PCEXP ',params[1].round(2))

Const  2876.68
1/PCEXP  -13557807.68


> ### Interpretação

Embora o modelo recíproco seja não linear em X, ele continua sendo linear nos parâmetros, portanto não há conflito com  o pressuposto de GaussMarkov de linearidade do modelo.

O parâmetro de B1 -13557807.68, sendo um valor negativo, indica que o valor de Y cresce a taxas decrescentes.

A constante tem uma aspacto importante no modelo recíproco por representar o valor máximo que a variável dependente pode assumir. Quanto X cresce indefinidamente, Y aproxima-se da constante do modelo. 

Não dífícil deduzir essa relação. Como estamos olhando para o inverso da variável X, ou seja 1/X, na media que X cresce, 1/X diminui. Se X tende ao infinito a relação 1/X tende a zero.

### $ Y= \beta_{0} +  \beta_{1} \frac{1}{X}  $

### $ \beta_{1} \frac{1}{X} = \lim_{x \to 0} \frac{1}{X} = 0$

### $ Y= \beta_{0} +  0  $
### $ Y= \beta_{0}$

Em resumo temos um modelo onde Y cresce a taxas decrescentes na medida em que X aumenta e tende ao valor 2876.68 na medida em que X cresce indefinidamente.

## Variáveis padronizadas

In [11]:
# Definir X e y
X = df['PCEXP_STD']
y = df['EXPDUR_STD']

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.params.round(4)

const        0.0000
PCEXP_STD    0.9827
dtype: float64

> ### Interpretação

A variação nesse caso é medida em termos de desvio-padrão. Portanto, se o regressor aumenta em média 1 desvio-padrão, o regressando irá aumentar $\beta_{1}$ unidades de desvio padrão.

A constante de uma regressão linear com dados padronizados sempre será zero. Lembre que $ \hat{\beta}_{0} = Y_{mean} + \hat{\beta}_{1}X_{mean} $. Lembre ainda que ao padronizarmos uma variável, sua média será igual a 0 e o desvio-padrão igual a 1. Como a variável Y está padronizada, a sua média é zero, logo, $ \hat{\beta}_{0} = 0 + \hat{\beta}_{1}X_{mean} $.

Trazendo para o nosso modelo, obervamos a constante com valor zero. O parâmetro da variável PCEXP_STD foi estimado em 0.9827. Isso significa que quando PCEXP_STD variar em média 1 desvio-padrão, a nossa variável target (EXPDUR_STD) irá variar 0.9827 unidade de desvio-padrão.

Uma das vantagens da padronização dos regressores é que colocamos todos na mesma base, portanto, podemos compará-los diretamente. Assim, podemos usar o valor dos parâmetros estimados como medida de força relativa dos vários regressores, ou seja, ao compararmos dois regressores com valores distintos, o que tiver maior valor tem maior poder explicativo.  

___

# Taxa de crescimento dos gastos com serviço

O objetivo agora é econtrar a taxa de crescimento dos gastos com serviço por trimestre de 2003 à 2006. 

O processo é simples. Aplicamos o Log à conhecida fóruma de juros compostos e pronto. Com pequenos ajustes à equação obtida teremos claramente um modelo de regressão linear simples.

Modelando a taxa de crescimento:

**Juros compostos**
### $ Y_t = Y_{0}(1 + i)^{t} $


**Aplicando ln aos Juros compostos**
### $ LnY_t = LnY_{0} + tLn(1 + i) $

#### $ \beta{0} =  LnY_{0}  $
#### $ \beta{1} =  Ln(1 + i)  $


**Substituindo e incluindo  termo de erro:**
## $ LnY_t = \beta_{0} +  \beta_{1}t + u{i}  $

Fica evidente que temos um modelo de regressão linear. Portanto, ao regredir o Log dos gastos contra o tempo, $\beta_1$ representará a taxa de crescimento média dos gastos ao longo do período observado. Já $\beta_0$ será o gasto do período anterior ao primeiro período observado, ou seja, $\beta_0$ será o gasto no pertíodo $t_0$.

In [12]:
# Definir X e y
X = (df['TIME'])
y = np.log(df['EXPSERVICES'])

# Modelo
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parametros
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            EXPSERVICES   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1273.
Date:                Fri, 02 Sep 2022   Prob (F-statistic):           1.50e-13
Time:                        16:56:18   Log-Likelihood:                 62.504
No. Observations:                  14   AIC:                            -121.0
Df Residuals:                      12   BIC:                            -119.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.3223      0.002   4900.714      0.000       8.319       8.326
TIME           0.0071      0.000     35.683      0.000       0.007       0.008
==============================================================================
Omnibus:                        1.355   Durbin-Watson:                   0.573
Prob(Omnibus):                  0.508   Jarque-Bera (JB):                0.838
Skew:                           0.195   Prob(JB):                        0.658
Kurtosis:                       1.867   Cond. No.                         18.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# Algumas transformações

# Obter a taxa de crescimento composta
tx_comp = np.exp(lin_reg.params[1]) - 1

# Obter a taxa de crescimento anual
tx_anual = (((1 + tx_comp)**4)-1)

# Obter o valor em t0
y0 = np.exp(lin_reg.params[0])

In [14]:
# View
print('Taxa Instantânea:', lin_reg.params[1].round(6))
print('Taxa composta:', tx_comp.round(6))
print('Taxa anual:', tx_anual.round(6))
print('Gasto em t0:', y0.round(2))

Taxa Instantânea: 0.007117
Taxa composta: 0.007142
Taxa anual: 0.028875
Gasto em t0: 4114.58


> ## Interpretação

O parâmetro estimado de TIME 0.7117% representa a taxa de crescimento instantânea trimestral dos gastos com serviços. Essa é ataxa de crescimento em um ponto do tempo e não a taxa de crescimento ao longo do tempo. 

A taxa de crescimento ao longo do tempo é a taxa de crescimento composta e podemos encontrá-la extraindo o anti-logarítimo da taxa de crescimento instantânea, como fizemos  em *tx_comp = np.exp(lin_reg.params[1]) - 1*. Assim obtemos uma taxa de 0.7142%, equivalente a uma taxa de crescimento anual de 2.88%.

O intercepto da regressão representa os gastos em t0, ou seja, os gastos ao final do último trimestre de 2002.

Neste último modelo trouxemos o sumário completo do modelo para chamar a atençao para o fato de que o parâmetro estimado está contido em um intervalo de confiança a um determinado nível confiança. Trazendo isso para a interpretação do intercepto, podemos extrair o anti-logarítimo do intervalo de confiança e observar a faixa de valores que o valor dos gastos em t0 pode assumir. 

In [15]:
# Extrair o intervalo de confiança do modelo e tomar o anti-logarítimo para obter o valor na escala original ($)
conf_Inf = np.exp(lin_reg.conf_int()[0:1].iloc[ : , 0].sum()).round(2)
conf_Sup = np.exp(lin_reg.conf_int()[0:1].iloc[ : , 1].sum()).round(2)

print('Lim inferior:', conf_Inf)
print('Lim superior:', conf_Sup)

Lim inferior: 4099.38
Lim superior: 4129.83


Portanto, o valor dos gastos ao fim do último trimestre de 2022, estatisticamente a um nível de confiança de 5%, é um valor entre 4099.38 e 4129.83. 

A interpretação com intervalos de confiança aplica-se a qualquer modelo que estimamos aqui.

## Modelo de tendência linear

Se regredirmos o valor em dólares dos gastos com serviço contra o tempo, teremos um modelo de tendência linear. Estamos falando de um modelo LinLin. 

In [16]:
# Set X and y
X = df['TIME']
y = df['EXPSERVICES']

# Model and fit
X_cte = sm.add_constant(X, has_constant = 'add')
lin_reg = sm.OLS(y, X_cte).fit()

# Parameters
lin_reg.params.round(2)

const    4110.66
TIME       30.84
dtype: float64

In [17]:
# Extrair o intervalo de confiança do modelo 
conf_Inf = lin_reg.conf_int()[0:1].iloc[ : , 0].sum().round(2)
conf_Sup = lin_reg.conf_int()[0:1].iloc[ : , 1].sum().round(2)

print('Lim inferior:', conf_Inf)
print('Lim superior:', conf_Sup)

# O valor do parâmetro estimado obviamente é o valor médio do intevalo de confiança
print('Constante:' , round((conf_Inf+conf_Sup)/2,2))

Lim inferior: 4096.02
Lim superior: 4125.29
Constante: 4110.66


> ## Interpretação

O parâmetro de TIME 30.84 está dizendo que os gastos com serviços aumentam em média a uma taxa absoluta de $30.84 bilhões por trimestre. O intercepto traz o valor dos gastos no fim do último trimeste de 2022.

Como utilizamos os dados na escala original, nenhuma tranformação foi necessária para obter os resultados. Comparando o modelo da taxa de crescimento com o modelo de tendência linear, vale destacar ainda quão próximos são os valores obtidos ao fim do último trimestre de 2002, tanto do parâmetro em si quanto os intervalos de confiança. 

___